# Análise de dados de encomendas depositadas no equipamento "Boca de Lobo"

Esta análise visa demonstrar as principais informações referentes às encomendas depositadas no equipamento "Boca de Lobo", instalado na estação do MetrôRio Estácio de Sá.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

O dataset é disponibilizado no formato xlsx pela aplicação.
A aplicação utiliza um banco SQLite e disponibiliza um arquivo xlsx com todos os registros desde a instalação da máquina.
A obtenção do arquivo xlsx é feito atráves do serviço ngrok

In [19]:
# @hidden_cell
url_boca_de_lobo = 'http://bocadelobo.sa.ngrok.io/public/completo'

In [20]:
%%time
encomendas_depositadas = pd.read_excel(url_boca_de_lobo)

HTTPError: HTTP Error 502: Bad Gateway

In [3]:
encomendas_depositadas

,Hash,Entregador,Entregadora,Cidade,UF,Data,Gaveta,QR,Código da encomenda
0,09668303d2a0a4cd17d753eed1a69fdb0ee09053,VVVGG,CN22,Rio de Janeiro,RJ,2020-09-22T12:30:33.418Z,1,NaN,VGYY
1,09668303d2a0a4cd17d753eed1a69fdb0ee09053,VVVGG,CN22,Rio de Janeiro,RJ,2020-09-22T12:30:33.418Z,1,NaN,TGGG
2,021a346c02719de2fb04fc422002e353eb30f1f2,TGF,CN22,Rio de Janeiro,RJ,2020-09-22T12:31:37.749Z,2,NaN,YYU
3,1347b82264fb6d841bfaca3b16ed75fd00ba495b,ALEX,B2W,Rio de Janeiro,RJ,2020-09-22T12:36:08.053Z,1,NaN,GF445CFGBR
4,1347b82264fb6d841bfaca3b16ed75fd00ba495b,ALEX,B2W,Rio de Janeiro,RJ,2020-09-22T12:36:08.053Z,1,NaN,TVGG65BR
...,...,...,...,...,...,...,...,...,...
122868,607d4eac7f3fa6d5f219319177672b993e400980,ANDRE,Total,Rio de Janeiro,RJ,2023-04-28T16:21:27.600Z,0,TXAU172100604tx,NaN
122869,607d4eac7f3fa6d5f219319177672b993e400980,ANDRE,Total,Rio de Janeiro,RJ,2023-04-28T16:21:27.600Z,0,AMZA712652160tx,NaN
122870,607d4eac7f3fa6d5f219319177672b993e400980,ANDRE,Total,Rio de Janeiro,RJ,2023-04-28T16:21:27.600Z,0,SGqrQbNlMc_001_v,NaN
122871,607d4eac7f3fa6d5f219319177672b993e400980,ANDRE,Total,Rio de Janeiro,RJ,2023-04-28T16:21:27.600Z,0,TXAU170468839tx,NaN


In [17]:
encomendas = encomendas_depositadas.filter(items=['Entregador', 'Entregadora', 'Cidade', 'UF', 'Data', 'Gaveta', 'Código da encomenda'])

NameError: name 'encomendas_depositadas' is not defined

In [5]:
encomendas = encomendas.rename(columns={'Código da encomenda':'Codigo'})

A data do sistema é disponibilizada com o fuso UTC 0. Além disso, é necessário converter o formato de data disponibilizado.

In [ ]:
data_formatada = []
erro_hora = timedelta(hours=3) #hora do sistema está considerando UTC 0. Conversão para UTC-3

for i in range(len(encomendas['Data'])):
  data_formatada.append((datetime(int(encomendas.Data[i][:4]), int(encomendas.Data[i][5:7]), int(encomendas.Data[i][8:10]), int(encomendas['Data'][i][11:13]), int(encomendas['Data'][i][14:16])) - erro_hora))


In [ ]:
encomendas['data'] = data_formatada

In [ ]:
encomendas.drop(columns={'Data'})

In [ ]:
encomenda_ano = encomendas.data.dt.year.value_counts().sort_index()
encomenda_ano.index

In [ ]:
x = encomenda_ano.index.astype(str)
height = encomenda_ano.values


bar_graph = plt.bar(x, height=height, color='red')
plt.xlabel("Ano")
plt.ylabel("Qnt de encomendas")
plt.gca().set_title('Encomendas por Ano')
plt.bar_label(bar_graph, fontsize=10)
plt.margins(x=0.01, y=0.1)
plt.show()

In [ ]:
plt.hist(encomendas.data.dt.hour, bins= 50, color='red')
plt.show()

In [ ]:
encomenda_hora = encomendas.data.dt.hour.value_counts().sort_index()

x_hora = encomenda_hora.index.astype(str)
height_hora = encomenda_hora.values


bar_graph_hora = plt.bar(x_hora, height=height_hora, color='red')
plt.xlabel("Hora")
plt.ylabel("Qnt de encomendas")
plt.gca().set_title('Encomendas por Horário')
plt.margins(x=0.01, y=0.1)
plt.show()

In [ ]:
entregadoras = encomendas.Entregadora.value_counts().to_frame()
entregadoras.head(20)

In [ ]:
#entregadoras.head(20).to_csv('entregadoras_rev1.csv', sep=';')

In [ ]:
x_empresas = entregadoras.head(20).index
width_empresas = entregadoras.head(20)['Entregadora']

bar_empresas = plt.barh(x_empresas, width=width_empresas)
plt.gca().invert_yaxis()
plt.gca().set_title('Encomendas por Empresas (20 maiores)')
plt.xlabel("Qnt de encomendas")
plt.ylabel("Encomendas")
plt.bar_label(bar_empresas, fontsize=10)
plt.margins(x=0.1, y=0.1)
plt.show()

In [ ]:
encomendas_depositadas.count()

In [ ]:
percent_qr = round(encomendas_depositadas.count().QR /  encomendas_depositadas.count().Hash,3) * 100
percent_noqr = round(100 - percent_qr, 3)


qr_control = pd.DataFrame.from_dict({"QR":percent_qr, "Teclado":percent_noqr}, orient='index')
qr_control

In [ ]:
qr_control.values.ravel()

In [ ]:
x_qr = qr_control.index
values_qr = qr_control.values.ravel()

bar_qr = plt.barh(x_qr, width=values_qr, color='red')
plt.gca().invert_yaxis()
plt.gca().set_title('Tipo de inserção')
plt.xlabel("Percentual")
plt.ylabel("Tipo")
plt.bar_label(bar_qr, fontsize=10)
plt.margins(x=0.1, y=0.1)
plt.show()

In [ ]:
dia = input("Dia pesquisado?")
mes = input("mes pesquisado?")
ano = input("ano pesquisado?")
dia, mes, ano

In [ ]:
dia_atual = datetime.now().date()
dia_atual

In [ ]:
encomendas_atual = encomendas[encomendas.data.dt.date == dia_atual]
encomendas_atual

In [ ]:
encomendas_atual['hora'] = encomendas_atual.data.dt.time
encomendas_atual = encomendas_atual.drop(columns={'Data'})
encomendas_atual

In [ ]:
encomendas_atual.to_csv('encomendas_atual.csv', sep=';')